# PRACTICE BUILDING TESTBEDS

This notebook poses a sequences of problems for building testbeds.

In [ ]:
if False:
    !pip uninstall numpy
    !pip install numpy==1.23.1
    !pip install tellurium
    !pip install controlSBML

## Preliminaries

In [ ]:
import controlSBML as ctl

import control
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tellurium as te
from IPython.display import Image
from IPython.core.display import HTML
ctl.__version__

# Helpers

In [ ]:
TEST_MODEL = """
species S1, S2, S3

S1 -> S2; k1*S1
S2 -> S3; k2*S2

S1 = 10
S2 = 0
S3 = 0
k1 = 1
k2 =2
"""
ctlsb = ctl.ControlSBML(TEST_MODEL, input_names=["S1", "S2"], output_names=["S3"])
TEST_SYS = ctlsb.makeNonlinearIOSystem("TEST_SYS")

## plotSys

In [ ]:
def plotSys(sys, times=None, initial_values=None, forced_input=None, title="", ax=None, output_labels=None, is_plot=True):
    """
    Plots the system.
    
    Parameters
    ----------
    sys: NonlinearIOSystem
    times: list-float (times for simulation)
    initial_values: list-float
    forced_input: list-float
    ax: matplotlib.Axes
    output_labels: list-str (names for output labels)
    
    Returns
    -------
    np.ndarray, np.ndarray, ax (times, predicted values, plot axis)
    """
    if times is None:
        times = np.linspace(0, 5, 100)
    if forced_input is None:
        ncol = len(times)
        nrow = len(sys.input_labels)
        forced_input = np.array([np.repeat(0, nrow) for _ in range(ncol)])
        forced_input = np.reshape(forced_input, (nrow, ncol))
    if initial_values is None:
        initial_values = ctl.makeStateVector(sys)
    if ax is None:
        _, ax = plt.subplots(1)
    if output_labels is None:
        output_labels = list(sys.output_labels)
    # Simulate the system
    t, y = control.input_output_response(sys, times, forced_input, initial_values)
    # Plot the response
    num_output = len(output_labels)
    if num_output == 1:
        if y.ndim == 1:
            plt.plot(t, y)
        else:
            plt.plot(t, y[0])
    else:
        for idx in range(num_output):
            ax.plot(t, y[idx])
    ax.legend(output_labels)
    if not is_plot:
        plt.close()
    return t, y, ax
    
# Tests
times = T = np.linspace(0, 70, 500)
times, predictions, _ = plotSys(TEST_SYS, times, is_plot=False)
assert(len(times) == len(predictions[0]))
print("OK!")

# Models

## Linear model

In [ ]:
LINEAR_MDL = """
S1 -> S2; k1*S1
S2 -> S3; k2*S2
S3 -> S4; k3*S3

k1 = 2
k2 = 1.5
k3 = 1
S1 = 10
S2 = 0
S3 = 0
S4 = 0
"""
LINEAR_RR = te.loada(LINEAR_MDL)
LINEAR_DATA = LINEAR_RR.simulate()
LINEAR_RR.plot(LINEAR_DATA)
LINEAR_STATE_NAMES = ["S1", "S2", "S3", "S4"]
LINEAR_PARAM_DCT = {"input_names": LINEAR_STATE_NAMES, "output_names": LINEAR_STATE_NAMES}

## WOLF model

In [ ]:
WOLF_CTLSB = ctl.ControlSBML("https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000206.2?filename=BIOMD0000000206_url.xml", 
                        input_names=["at"], output_names=["s5"])
WOLF_RR = WOLF_CTLSB.roadrunner
WOLF_RR.simulate(0, 1, 100)
WOLF_RR.plot()

In [ ]:
WOLF_ANT = WOLF_RR.getAntimony()
print(WOLF_ANT)

# Review: Closed loop testbed for Wolf

In [ ]:
Image("wolf_cls1.png", width = 600, height = 300)

This example explores the use of feedback control to regulate the concentration of pyruvate in
a model of glycolytic oscillations (BioModels 206).

Pyruvate is ``s5`` in the model.
It is regulated using ATP, species ``at`` in the model.

Below is the behavior of the unregulated system.
Note the oscillations in ``s5``.

In [ ]:
# Simulate the open loop system with no change in its input (at)
wolf_sys = WOLF_CTLSB.makeNonlinearIOSystem("wolf_sys")
ts = ctl.simulateSystem(wolf_sys, end_time=30)
_ = ctl.plotOneTS(ts, figsize=(5,5), ylim=[0, 10])

**Control Objectives**

1. Eliminate the oscillations from ``s5``
1. Coverge to 6.

Next, we construct a PI controller.

In [ ]:
S5_SETPOINT = 6

In [ ]:
def makePIController(setpoint=S5_SETPOINT, kP=20, kI=10):
    """
    Makes a PI controller
    
    Parameters
    ----------
    setpoint: float
    kP: float
    kI: float
    
    Returns
    -------
    NonlinearIOSystem
    """
    # Calculate derivative of state
    def updfcn(t, x, u, _):
        # Accumulate the control error
        # t: float (time)
        # x: array-float (state)
        # u: array-float (output from OLS)
        # returns: array-float (derivative of state)
        dx = setpoint - u[0]
        return dx

    # Calculate output value
    def outfcn(t, x, u, _):
        # Calculate the output from the input
        # t: float (time)
        # x: array-float (state)
        # u: array-float (inputs)
        # returns: array (output)
        new_err = setpoint - u[0]
        output = kI*x[0] + kP*new_err
        return output

    controller = control.NonlinearIOSystem(
      updfcn,
      outfcn,
      states=1,
      inputs=['in'],
      outputs=['out'], name='controller_sys')
    return controller

# Tests
controller = makePIController()
assert("NonlinearIOSystem" in str(type(controller)))
print("OK")

We form a closed loop system so that the output of the glycolytic system, ``sys.s5``, is
the input the the controller, ``controller.in``.
And, the output of the controller, ``controller.out``, is the input the glycolytic system.

In [ ]:
def makeInterconnect(wolf_sys=wolf_sys, **kwargs):
    """
    Create an Interconnect object for the Wolf closed loop system.
    
    Parameters
    ----------
    wolf_sys: NonlinearIOSystem
    kwargs: dict (arguments for makePIController)
    
    Returns
    -------
    Interconnect
    """
    controller_sys = makePIController(**kwargs)
    closed_sys = control.interconnect(
      [wolf_sys, controller_sys],       # systems
      connections=[
        ['wolf_sys.at', 'controller_sys.out'],
        ['controller_sys.in',  'wolf_sys.s5'],
      ],
      inplist=["controller_sys.in"],
      outlist=["wolf_sys.s5"],
    )
    return closed_sys

# Tests
interconnect = makeInterconnect()
assert("Interconnect" in str(type(interconnect)))
print("OK!")

Now, we simulate the closed loop system.
Note that we have eliminated oscillations, and ``s5`` converges to 6.

In [ ]:
closed_sys = makeInterconnect(kP=100, kI=10)
end_time = 30
ts = ctl.simulateSystem(closed_sys, end_time=end_time, output_names=["s5"])
ts["setpoint"] = S5_SETPOINT
# Plot the resultkkk
_ = ctl.plotOneTS(ts, figsize=(5,5), ylim=[0, 10], title="Closed Loop Wolf System")

We see that the control system has achieved its objectives.

# Closed loop with explicit differencing

The Wolf testbed is currently this.

In [ ]:
Image("wolf_cls1.png", width = 600, height = 300)

We want to change it to this.

In [ ]:
Image("wolf_cls.png", width = 600, height = 300)

Considerations.
1. We need to create a no system, the summation node.
1. The closed loop system will have an input, the setpoint.

## Pure controller

The controller should only take control error as input.

In [ ]:
def makePurePIController(kP=20, kI=10):
    """
    Makes a PI controller with control error as input
    
    Parameters
    ----------
    kP: float
    kI: float
    
    Returns
    -------
    NonlinearIOSystem
    """
    # Calculate derivative of state
    def updfcn(t, x, u, _):
        # Accumulate the control error
        # t: float (time)
        # x: array-float (state)
        # u: array-float (control error)
        # returns: array-float (derivative of state)
        return u[0]

    # Calculate output value
    def outfcn(t, x, u, _):
        # Calculate the output from the input
        # t: float (time)
        # x: array-float (state)
        # u: array-float (inputs)
        # returns: array (output)
        output = kI*x[0] + kP*u[0]
        return output

    controller = control.NonlinearIOSystem(
      updfcn,
      outfcn,
      states=1,
      inputs=['in'],
      outputs=['out'], name='controller_sys')
    return controller

# Tests
kP = 10
kI = 20
controller_sys = makePurePIController(kP=kP, kI=kI)
assert("NonlinearIOSystem" in str(type(controller)))
times, data,_ = plotSys(controller_sys, forced_input=[1], is_plot=False)
assert(data[0] == kP)
assert(np.isclose(data[-1], kP + times[-1]*kI))
print("OK")

## Differencing Node

In [ ]:
Image("differ_sys.png", width = 800, height = 400)

Key questions about ``differ_sys``:
1. How many inputs?
1. How many outputs?
1. What if any state is there, and how is it calculated?
1. How is the output calculated from the state and the input?

In [ ]:
def outfcnDiffer(t, x, u, _):
    # Calculate the output from the input
    # t: float (time)
    # x: array-float (state)
    # u: array-float (inputs)
    # returns: array (output)
    return u[0] - u[1]

In [ ]:
differ_sys = control.NonlinearIOSystem(
      None,
      outfcnDiffer,
      inputs=['setpoint', 'measurement'],
      outputs=['out'], name='differ_sys')

In [ ]:
# Simulate to verify 
_ = plotSys(differ_sys, forced_input=[3, 2])

In [ ]:
def makeDiffer():
    """
    Makes a system that constructs differences.
    
    Returns
    -------
    NonlinearIOSystem
    """
    def outfcnDiffer(t, x, u, _):
        # Calculate the output from the input
        # t: float (time)
        # x: array-float (state)
        # u: array-float (inputs)
        # returns: array (output)
        return u[0] - u[1]
    
    return control.NonlinearIOSystem(
          None,
          outfcnDiffer,
          inputs=['setpoint', 'measurement'],
          outputs=['out'], name='differ_sys')

# Tests
differ_sys = makeDiffer()
assert("NonlinearIOSystem" in str(type(differ_sys)))
# Simulate to verify 
_, data, __ = plotSys(differ_sys, forced_input=[3, 2], is_plot=False)
assert(np.var(data[0]) == 0)
assert(data[0][0] == 1)
print("OK")

## Adder

There are potentially several places where we need to add signals, at least one with
differencing. Can we generalize the Differ so:
1. Aribtrary number of inputs.
1. Inputs may be subtracted or added.

In [ ]:
def makeAdder(input_names, output_name="out", system_name="adder_sys"):
    """
    Creates a NonlinearIOSystem that does addition
    of the inputs. Any name that begins wit a "-" is subtracted.
    
    Parameters
    ----------
    input_nmames: list-str
    output_name: str
    system_name: str
    
    Returns
    -------
    NonlinearIOSystem
    """
    adjustments = []
    adjusted_input_names = []
    def outfcn(_, __, u, ___):
        # Calculate the output from the input
        # t: float (time)
        # x: array-float (state)
        # u: array-float (inputs)
        # returns: array (output)
        return adjustments.dot(u)
    #
    for name in input_names:
        if name[0] == "-":
            value = -1
            new_name = name[1:]
        else:
            value = 1
            new_name = name
        adjustments.append(value)
        adjusted_input_names.append(new_name)
    adjustments = np.array(adjustments)
    #
    return control.NonlinearIOSystem(
          None,
          outfcn,
          inputs=adjusted_input_names,
          outputs=output_name, name=system_name)

# Tests
adder_sys = makeAdder(["a", "-b", "c"])
assert("NonlinearIOSystem" in str(type(adder_sys)))
# Simulate to verify 
_, data, __ = plotSys(adder_sys, forced_input=[3, 2, 4], is_plot=False)
assert(np.var(data[0]) == 0)
assert(data[0][0] == 5)
print("OK")

## Connections

In [ ]:
def makeInterconnectWithDiffer(wolf_sys=wolf_sys, **kwargs):
    """
    Create an Interconnect object for the Wolf closed loop system.
    
    Parameters
    ----------
    wolf_sys: NonlinearIOSystem
    kwargs: dict (arguments for makePIController)
    
    Returns
    -------
    Interconnect
    """
    controller_sys = makePurePIController(**kwargs)
    differ_sys = makeAdder(["setpoint", "-measurement"], system_name="differ_sys")
    closed_sys = control.interconnect(
      [wolf_sys, controller_sys, differ_sys],       # systems
      connections=[
        ['differ_sys.measurement', "wolf_sys.s5"], 
        ['wolf_sys.at', 'controller_sys.out'],
        ['controller_sys.in',  'differ_sys.out'],
      ],
      inplist=["differ_sys.setpoint"],
      outlist=["wolf_sys.s5"],
    )
    return closed_sys

# Tests
setpoint = 6
new_closed_sys = makeInterconnectWithDiffer(kP=20, kI=10)
assert("Interconnect" in str(type(new_closed_sys)))
_, data, ax = plotSys(new_closed_sys, forced_input=[setpoint], times=np.linspace(0, 20, 100), 
                      is_plot=False,
                      output_labels=["s5"])
assert(data[0] == 8)
assert(np.abs(setpoint - data[-1]) < 0.2)
print("OK!")

## Explorations

In [ ]:
# Compare controller designs
setpoint = 6
pairs = [(1,1), (5,5), (20, 10), (20, 20), (50, 50)] # kP, kI
_, ax = plt.subplots(1, figsize=(15, 5))
times = np.linspace(0, 10, 100)
legends = []
for idx, (kP, kI) in enumerate(pairs):
    title = "kP=%2.2f kI=%2.2f" % (kP, kI)
    legends.append(title)
    new_closed_sys = makeInterconnectWithDiffer(kP=kP, kI=kI)
    _ = plotSys(new_closed_sys, forced_input=[setpoint], times=times,
                      output_labels=["s5"], ax=ax)
plt.plot([times[0], times[-1]], [setpoint, setpoint], linestyle="--")
plt.legend(legends)
plt.xlabel("time")
plt.title("Comparisons of closed loop designs")

# Exercise: Noise and Filters

In [ ]:
Image("add_noise_filter.png", width = 600, height = 450)

1. Create a function that creates a ``NonlinearIOSystem`` for the filter. Its state is the averaged
measure output. Its state equation is 
$\dot{x}(t) = k_f u(t) - k_f x(t)$, where $k_f > 0$.
It's output equation is $y(t) = x(t)$.
Make sure that you have a test for this function that simulates the ``NonlinearIOSystem``.
1. Create a function that creates a ``NonlinearIOSystem`` that generates a sine wave with frequency $f$ and amplitude $A$. (What are its inputs? What are its outputs?)
Make sure that you have a test for this function that simulates the ``NonlinearIOSystem``.
1. Construct a testbed for the closed loop system using the above architecture with
the Wolf model as the open loop system using ``at`` as input and ``s5`` as output.
1. Tune the controller and filter for a setpoint of 6, settling time of 30 s, and minimal oscillations.